# Experimentos

En este archivo está el código para correr los experimentos y guardar los resultados en la carpeta /resultados como archivos.txt
- Adicionalmente, se utilizaran archivos .csv para tener un censo de los tiempos de ejecución de cada algoritmo y experimento, junto con información útil 

In [1]:
import math, subprocess
import numpy as np
import pandas as pd

import os
import shutil
from IPython.display import display, clear_output

Funciones útiles para los experimentos

In [2]:
# Creación de la carpeta resultados

def crear_carpeta(dir):
    if os.path.exists(dir) : shutil.rmtree(dir)
    os.makedirs(dir)

A continuación leemos los datasets en dataframes de Pandas

In [10]:
p_ext_alta_temperatura     = pd.read_csv("instancias/D1/indice.csv")
temperatura_cambiante      = pd.read_csv("instancias/D2/indice.csv")
refrigeracion_no_funcional = pd.read_csv("instancias/D3/indice.csv")
sensores_fallados          = pd.read_csv("instancias/D4/indice.csv")

La siguiente función sirve para correr el código sobre una instancia ejecutando un método en particular

- 0 : Gauss
- 1 : LU


In [11]:
def ejecutar_algoritmo(archivo_entrada, archivo_salida, metodo):
    # Crear proceso para ejecutar el codigo. 
    process = subprocess.run(["../src/ejecutable", archivo_entrada, archivo_salida, metodo], stderr=subprocess.PIPE, stdout=subprocess.PIPE, universal_newlines = True, encoding="ascii")

In [13]:
# Instancias, métodos y experimentos
instancias = ["D1", "D2", "D3", "D4"]
metodos = ["Gauss", "LU"]
experimentos = ["EXP1", "EXP2", "EXP3A", "EXP3B"]

In [14]:
# Creación de las carpetas de resultados
# También sirve para "resetear" las corridas.

for exp in experimentos:
    dir = F'resultados/{exp}'
    crear_carpeta(dir)


## Ejecución de los algoritmos

In [7]:
def aplicar_metodo(archivo_entrada, archivo_salida, metodo):
    process = subprocess.run(["../src/ejecutable", archivo_entrada, archivo_salida, metodo], stderr=subprocess.PIPE, stdout=subprocess.PIPE, universal_newlines = True, encoding="ascii")
    result = process.stderr.split('\n')
    result.pop()
    result = [float(x) for x in result]
    return result

## Experimento 1 - Prueba de modelo

En este experimento se pretende encontrar un balance entre la cantidad de ángulos y radios, y los tiempos de cómputo de los algoritmos para poder distinguir la ubicación de la isoterma de forma nítida. 
> Este experimento utilizará el `Dataset 1 : Pared exterior de alta temperatura`

In [16]:
experimento = []

for i in range(0,len(p_ext_alta_temperatura)):
    fila_i = p_ext_alta_temperatura.iloc[i]
    experimento.append([fila_i["Dataset"], fila_i["Nombre de instancia"], fila_i["m+1"], fila_i["n"], fila_i["Archivo"], fila_i["Radio interno"], fila_i["Radio externo"]])

In [ ]:
columnas = ["Dataset", "Nombre de instancia", "m+1", "Media", "Desvio", "Radio interno", "Radio externo"]
filas = []

T = 5 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).
# Ejecutamos el experimento T veces y obtenemos la mediana.

numero = 1
for inst in experimento:
    clear_output(wait=True)
    tiempos = []
    for i in range(0,T):
        tiempos.append(aplicar_metodo(inst[4],F"resultados/EXP1/{inst[1]}-resultado.out", F"0"))
    media = np.median(tiempos, axis = 0)
    desvio = np.std(tiempos, axis = 0)
    filas.append([inst[0], inst[1], inst[2], inst[3], media[0], desvio[0], ])
    display('Experimento: ' + str(numero) + "/" + str(len(experimento)))
    display(inst[0] + "/" + inst[1])
    numero = numero + 1
    
df_resultado = pd.DataFrame(filas, columns=columnas);    
df_resultado.to_csv("resultados/EXP1/resultado.csv", index=False, header=True);

## Experimento 2 - Prueba de cómputo

Una vez encontrada una discretización apropiada por el `Experimento 1 - Prueba de modelo`, se utilizará la misma para comparar los tiempos de cómputo entre la `Eliminación Gaussiana` y la `Factorización LU`, donde se tiene que utilizar un mismo sistema repetidas veces. (Este experimento hará uso de un dataset que considera el error posible al momento de sensar las temperaturas del horno).
> Este experimento utilizará el `Dataset 2 : Temperaturas que cambian con el tiempo`

In [8]:
fila_n = temperatura_cambiante.iloc[-1]
experimento = [fila_n["Dataset"], fila_n["Nombre de instancia"], fila_n["Número de instancia"], fila_n["Archivo"]]

In [10]:
columnas = ["Dataset", "Número de instancia", "Método", "Media", "Desvio"]
filas = []

T = 5 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).
# Ejecutamos el experimento T veces y obtenemos la mediana.
tiempos = []
for i in range(0,T):
    tiempos.append(aplicar_metodo(experimento[3],F"resultados/EXP2/{experimento[1]}-resultado.out", F"0"))
media = np.median(tiempos, axis = 0)
desvio = np.std(tiempos, axis = 0)
for j in range(0, len(media)):
    filas.append([experimento[0],j+1, F"0", media[j], desvio[j]])

tiempos = []
for i in range(0,T):
    tiempos.append(aplicar_metodo(experimento[3],F"resultados/EXP2/{experimento[1]}-resultado.out", F"1"))
media = np.median(tiempos, axis = 0)
desvio = np.std(tiempos, axis = 0)
for j in range(0, len(media)):
    filas.append([experimento[0],j+1, F"1", media[j], desvio[j]])
    
df_resultado = pd.DataFrame(filas, columns=columnas);    
df_resultado.to_csv("resultados/EXP2/resultado.csv", index=False, header=True);

## Experimento 3A - Prueba de riesgo

Utilizando los hallazgos del `Experimento 1 - Prueba de modelo` y `Experimento 2 - Prueba de cómputo`, se quiere simular un escenario donde una porción del sistema de refrigeración del horno no funciona como se debe, de forma que mitad del horno incrementa su temperatura a través del tiempo, potencialmente generando una situación de riesgo. Con esto se pretende ver como la isoterma se va deformando a medida que este escenario progresa.
> Este experimento utilizará el `Dataset 3 : Horno con sistema de refrigeración no funcional`

In [34]:
fila_n = refrigeracion_no_funcional.iloc[-1]
experimento = [fila_n["Dataset"], fila_n["Nombre de instancia"], fila_n["Número de instancia"], fila_n["Archivo"]]

In [35]:
ejecutar_algoritmo(experimento[3],F"resultados/EXP3A/{experimento[1]}-resultado.out", F"1")

## Experimento 3B - Prueba de sensado

Utilizando los hallazgos del `Experimento 1 - Prueba de modelo` y `Experimento 2 - Prueba de cómputo`, se quiere simular un escenario donde un conjunto de los sensores son defectuosos y no miden los valores adecuados. Con esto se pretende ver como la isoterma se va deformando en dichos sensores con cada instancia.
> Este experimento utilizará el `Dataset 4 : Horno con sensores fallados`

In [ ]:
fila_n = sensores_fallados.iloc[-1]
experimento = [fila_n["Dataset"], fila_n["Nombre de instancia"], fila_n["Número de instancia"], fila_n["Archivo"]]

In [ ]:
ejecutar_algoritmo(experimento[3],F"resultados/EXP3B/{experimento[1]}-resultado.out", F"1")